In [1]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
#import random
#import sklearn
#import collections
#from sklearn.model_selection import train_test_split
#import json
#import pylab
#from scipy.optimize import curve_fit
from tensorflow.keras import layers, Model
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import load_model
from sklearn.metrics import roc_curve, auc
import matplotlib
import matplotlib.patches as mpatches
#import shap
#import pandas as pd
import tensorflow as tf
#import tarfile
from tensorflow.keras.models import load_model
#from qkeras import QActivation, QDense, QConv2D, QBatchNormalization
import ensembler_functions as ef
import tf2onnx
import onnx
import os
from sklearn.model_selection import train_test_split
#import load_and_match as lam

2025-02-19 22:50:41.699986: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-19 22:50:41.772869: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-19 22:50:45.279219: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Set matplotlib default color cycle
new_color_cycle = [
    '#1f77b4',
    '#ff7f0e',
    '#2ca02c',
    '#d62728',
    '#9467bd',
    '#8c564b',
    '#e377c2',
    '#7f7f7f',
    '#bcbd22',
    '#17becf',
    '#aec7e8',
    '#ffbb78',
    '#98df8a',
    '#ff9896',
    '#c5b0d5',
    '#c49c94',
    '#f7b6d2',
    '#c7c7c7',
    '#dbdb8d',
    '#9edae5'
]

# You can then apply this new color cycle to your matplotlib plots
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=new_color_cycle)

In [3]:
L1AD_rate = 1000
target_rate = 10

In [ ]:
# First, one should run run_lam.py to generate match L1AD scores to the events, and then set that path
# in ensembler_functions load_and_preprocess

In [5]:
# Success! These AD scores are the same.

### Main (now using the folded model)

In [4]:
data_info = {
    "train_data_scheme": "B+C_loose", 
    "pt_normalization_type": "global_division", 
    "L1AD_rate": 1000, 
    "pt_thresholds": [50, 30, 30, 30], 
    "pt_scale_factor": 0.05,
    "comments": "No overlap removal, train over B+C with a loose threshold"
}

training_info = {
    "save_path": "/eos/home-m/mmcohen/ad_trigger_development/trained_models/trial_127", 
    "dropout_p": 0.1, 
    "L2_reg_coupling": 0.01, 
    "latent_dim": 4, 
    "large_network": True, 
    "num_trainings": 10, 
    "training_weights": True,
    "obj_type": 'HLT',
    "overlap_removal": False
}

#datasets, data_info = ef.load_and_preprocess(**data_info)

In [5]:
#training_info, data_info = ef.train_multiple_models(datasets, data_info, **training_info)
#del datasets

Booting up... initializing trainings of 10 models

starting training model 0...
20611/20611 ━━━━━━━━━━━━━━━━━━━━ 19s 905us/step
3638/3638 ━━━━━━━━━━━━━━━━━━━━ 3s 876us/step

Epoch 16: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
model 0 success

starting training model 1...
20611/20611 ━━━━━━━━━━━━━━━━━━━━ 19s 899us/step
3638/3638 ━━━━━━━━━━━━━━━━━━━━ 3s 859us/step

Epoch 13: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
model 1 success

starting training model 2...
20611/20611 ━━━━━━━━━━━━━━━━━━━━ 19s 892us/step
3638/3638 ━━━━━━━━━━━━━━━━━━━━ 3s 840us/step

Epoch 18: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 32: ReduceLROnPlateau reducing learning rate to 1.0000000474974514e-05.
model 2 success

starting training model 3...
20611/20611 ━━━━━━━━━━━━━━━━━━━━ 19s 887us/step
3638/3638 ━━━━━━━━━━━━━━━━━━━━ 3s 880us/step

Epoch 9: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
model 3 success


2025-02-19 21:38:05.856116: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-19 21:38:05.919143: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-19 21:38:05.919508: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [ ]:
datasets, l1Seeded_results, l1All_results = ef.process_multiple_models(
    training_info=training_info,
    data_info=data_info,
    plots_path=training_info['save_path']+'/plots',
    target_rate=target_rate,
    L1AD_rate=L1AD_rate,
    obj_type=training_info['obj_type']
)

powering on... preparing to run evals
Loaded A14N23LO from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/02-03-2025/A14N23LO.h5
Loaded EB_473255 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/02-03-2025/EB_473255.h5
Loaded EB_475321 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/02-03-2025/EB_475321.h5
Loaded EB_482596 from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/02-03-2025/EB_482596.h5
Loaded HAHMggfZdZd2l2nu from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/02-03-2025/HAHMggfZdZd2l2nu.h5
Loaded HHbbttHadHad from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/02-03-2025/HHbbttHadHad.h5
Loaded HLT_noalg_eb_L1All from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_matched_ntuples/02-03-2025/HLT_noalg_eb_L1All.h5
Loaded ZZ4lep from /eos/home-m/mmcohen/ad_trigger_development/data/loaded_and_m

/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 78 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


137150/137150 ━━━━━━━━━━━━━━━━━━━━ 225s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12412/12412 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12436/12436 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12425/12425 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12459/12459 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12428/12428 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
12421/12421 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12464/12464 ━━━━━━━━━━━━━━━━━━━━ 20s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
167097/167097 ━━━━━━━━━━━━━━━━━━━━ 273s 2ms/step
model 0:
L1AD_pure_rate: 993.9470406630285
L1AD_total_rate: 11413.139244071019
L1AD_threshold: 7.1612677574157715
l1Seeded:::
HLTAD_pure_rate: 2.3475064132002967
HLTAD_total_

/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/eos/home-i03/m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/ensembler_functions.py:1498: RuntimeWarning: divide by zero encountered in scalar divide
  return x / (1 - x)
/eos/home-i03/m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/ensembler_functions.py:1546: RuntimeWarning: invalid value encountered in scalar subtract
  uncertainties_up[tag] = func(eff + eff_unc_up) - func(eff)
/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/matplotlib/axes/_axes.py:3683: RuntimeWarning: invalid value encountered in scalar add
  low, high = dep + np.vstack([-(1 - lolims), 1 - uplims]) * err


l1All:::
HLTAD_pure_rate: 6.707942929661202
HLTAD_total_rate: 8.904841331261933
HLTAD_threshold: 76.02272033691406

ROC threshold for tag HLT_noalg_eb_L1All was given to be 76.02272033691406
Corresponding ROC FPR is 8.904841331268145e-05
ROC threshold for tag mc23e_ChiPlusChiMinus100_99_0p3ns was given to be 76.02272033691406
Corresponding ROC FPR is 8.904841331268145e-05
ROC threshold for tag mc23e_ChiPlusChiMinus500_40_10ns was given to be 76.02272033691406
Corresponding ROC FPR is 8.904841331268145e-05
ROC threshold for tag mc23e_HNLeemu was given to be 76.02272033691406
Corresponding ROC FPR is 8.904841331268145e-05
ROC threshold for tag mc23e_RS_G_hh_bbbb_AutoWidth_c10_M300 was given to be 76.02272033691406
Corresponding ROC FPR is 8.904841331268145e-05
ROC threshold for tag mc23e_jjJZ2 was given to be 76.02272033691406
Corresponding ROC FPR is 8.904841331268145e-05
ROC threshold for tag mc23e_jjJZ4 was given to be 76.02272033691406
Corresponding ROC FPR is 8.904841331268145e-05
R

...

l1All:::
HLTAD_pure_rate: 1.9399739796530566
HLTAD_total_rate: 2.483846476932652
HLTAD_threshold: 178.4081268310547

ROC threshold for tag HLT_noalg_eb_L1All was given to be 178.4081268310547
Corresponding ROC FPR is 2.4838464769346342e-05
ROC threshold for tag mc23e_ChiPlusChiMinus100_99_0p3ns was given to be 178.4081268310547
Corresponding ROC FPR is 2.4838464769346342e-05
ROC threshold for tag mc23e_ChiPlusChiMinus500_40_10ns was given to be 178.4081268310547
Corresponding ROC FPR is 2.4838464769346342e-05
ROC threshold for tag mc23e_HNLeemu was given to be 178.4081268310547
Corresponding ROC FPR is 2.4838464769346342e-05
ROC threshold for tag mc23e_RS_G_hh_bbbb_AutoWidth_c10_M300 was given to be 178.4081268310547
Corresponding ROC FPR is 2.4838464769346342e-05
ROC threshold for tag mc23e_jjJZ2 was given to be 178.4081268310547
Corresponding ROC FPR is 2.4838464769346342e-05
ROC threshold for tag mc23e_jjJZ4 was given to be 178.4081268310547
Corresponding ROC FPR is 2.48384647693463

/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/sklearn/metrics/_ranking.py:1183: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


ROC threshold for tag kaito was given to be 178.4081268310547
Corresponding ROC FPR is 2.4838464769346342e-05
phase 1: starting evals of model 9...


/cvmfs/sft.cern.ch/lcg/views/LCG_106a_cuda/x86_64-el9-gcc11-opt/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 78 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


137150/137150 ━━━━━━━━━━━━━━━━━━━━ 238s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 22s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12412/12412 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12436/12436 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12425/12425 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12459/12459 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12428/12428 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12421/12421 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
12464/12464 ━━━━━━━━━━━━━━━━━━━━ 21s 2ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
167097/167097 ━━━━━━━━━━━━━━━━━━━━ 285s 2ms/step


In [ ]:
idx = 32541
print(datasets['EB_test']['HLT_data'][idx])
print(datasets['EB_test']['HLT_preprocessed_data'][idx])

In [7]:
#ef5.save_subdicts_to_h5(datasets, '/eos/home-m/mmcohen/ntuples/dataset_with_AD_scores')

Saved HLT_noalg_eb_L1All to /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/HLT_noalg_eb_L1All.h5
Saved mc23e_ChiPlusChiMinus100_99_0p3ns to /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/mc23e_ChiPlusChiMinus100_99_0p3ns.h5
Saved mc23e_ChiPlusChiMinus500_40_10ns to /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/mc23e_ChiPlusChiMinus500_40_10ns.h5
Saved mc23e_HNLeemu to /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/mc23e_HNLeemu.h5
Saved mc23e_RS_G_hh_bbbb_AutoWidth_c10_M300 to /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/mc23e_RS_G_hh_bbbb_AutoWidth_c10_M300.h5
Saved mc23e_ttbar_1lep to /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/mc23e_ttbar_1lep.h5
Saved mc23e_ttbar_2lep to /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/mc23e_ttbar_2lep.h5
Saved physMain_474448GRLReal to /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/physMain_474448GRLReal.h5
Saved physMain_474679GRLReal to /eos/home-m/mmcohen/ntuples/dataset_with_AD_scores/physMain_474679GRLReal.

In [9]:
for tag, data_dict in datasets.items():
    print(f'{tag}:')
    for key, value in data_dict.items():
        print(f'  {key}: {np.array(value).shape}')

HLT_noalg_eb_L1All:
  HLT_data: (1097197, 48)
  L1_data: (1097197, 48)
  event_numbers: (1097197,)
  passHLT: (1097197,)
  passL1: (1097197,)
  pileups: (1097197,)
  run_numbers: (1097197,)
  topo2A_AD_scores: (1097197,)
  weights: (1097197,)
  HLT_preprocessed_data: (1097197, 48)
  HLT_model_outputs: (1097197, 48)
  HLT_latent_reps: (1097197, 4)
  HLT_AD_scores: (1097197,)
  calculated_HLT_AD_scores: (1097197,)
  region_labels: (1097197,)
mc23e_ChiPlusChiMinus100_99_0p3ns:
  HLT_data: (100000, 48)
  L1_data: (100000, 48)
  event_numbers: (100000,)
  passHLT: (100000,)
  passL1: (100000,)
  pileups: (100000,)
  run_numbers: (100000,)
  topo2A_AD_scores: (100000,)
  weights: (100000,)
  HLT_preprocessed_data: (100000, 48)
  HLT_model_outputs: (100000, 48)
  HLT_latent_reps: (100000, 4)
  HLT_AD_scores: (100000,)
  calculated_HLT_AD_scores: (100000,)
  region_labels: (100000,)
mc23e_ChiPlusChiMinus500_40_10ns:
  HLT_data: (100000, 48)
  L1_data: (100000, 48)
  event_numbers: (100000,)
  

In [7]:
# # Convert to ONNX
ef5.convert_to_onnx(
    training_info=training_info, 
    data_info=data_info, 
    model_version=0, 
    save_dir='/eos/home-m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/onnx_models', 
    opset=13, 
    input_dim=48
)

/cvmfs/sft.cern.ch/lcg/views/LCG_106a_swan/x86_64-el9-gcc13-opt/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 78 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional_31"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_12 (InputLayer)     │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_21 (Functional)      │ (None, 48)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_29 (Functional)      │ (None)                 │        49,780 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49,780 (194.45 KB)

 Trainable params: 48,588 (189.80 KB)

 Non-trainable params: 1,192 (4.66 KB)

ONNX HLT_AE model saved to: /eos/home-m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/onnx_models/folded_MSE_AE_0.onnx


2025-01-23 21:57:50.986175: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2025-01-23 21:57:50.986520: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2025-01-23 21:57:51.994756: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2025-01-23 21:57:51.995036: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


In [8]:
datasets = ef5.compare_tf_with_onnx(
    datasets, 
    training_info, 
    data_info, 
    model_version=0, 
    onnx_path='/eos/home-m/mmcohen/ad_trigger_development/src/AD_trigger_training/HLTAD/onnx_models'
)

In [9]:
idx=1001
print(datasets['EB_test']['HLT_AD_scores'][idx:idx+5])
print(datasets['EB_test']['ONNX_HLT_AD_scores'][idx:idx+5])

[3.0919242 2.3782115 0.        2.1444201 0.       ]
[3.0919242 2.3782115 0.        2.1444201 0.       ]


In [10]:
print(datasets['EB_test'].keys())

dict_keys(['HLT_data', 'L1_data', 'event_numbers', 'passHLT', 'passL1', 'pileups', 'run_numbers', 'topo2A_AD_scores', 'weights', 'HLT_preprocessed_data', 'HLT_model_outputs', 'HLT_latent_reps', 'HLT_AD_scores', 'calculated_HLT_AD_scores', 'region_labels', 'ONNX_HLT_AD_scores'])


In [11]:
print(datasets['EB_test']['HLT_data'][idx])
print(datasets['EB_test']['HLT_preprocessed_data'][idx])
print(datasets['EB_test']['HLT_model_outputs'][idx])
print(datasets['EB_test']['HLT_AD_scores'][idx])
print(datasets['EB_test']['calculated_HLT_AD_scores'][idx])

[ 1.80151749e+02  3.79687846e-01  1.12414896e+00  1.67376053e+02
  1.35481119e+00 -1.77274609e+00  2.84032555e+01 -3.41898251e+00
  1.67734385e-01  2.69995956e+01 -3.85731995e-01 -3.04468894e+00
  2.01521053e+01  1.11932135e+00  2.04924083e+00  1.99110374e+01
  2.58848047e+00 -8.71628642e-01  7.17338181e+01  4.16264683e-01
  1.11409879e+00  7.17338181e+01  4.16264683e-01  1.11409879e+00
  1.52860832e+01  1.32013667e+00 -1.71287799e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  7.24524918e+01  3.84140939e-01  1.11734807e+00  7.24524918e+01
  3.84140939e-01  1.11734807e+00  7.24524918e+01  3.84140939e-01
  1.11734807e+00  2.46440144e+01  0.00000000e+00 -2.80366850e+00]
[ 9.0075874e+00  3.7968785e-01 -2.3841858e-07  8.3688030e+00
  1.3548112e+00 -2.8968952e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
  0.0000000e+00  0.0000000e+00  0.00

In [ ]:
# As we can see, the calculated MSE is the same as the MSE predicted by the folded-in model!